In [1]:
from sentence_transformers import SentenceTransformer
from transformers import GPT2Model, GPT2Tokenizer
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from tqdm.notebook import tqdm_notebook
from transformers import BertTokenizer, BertModel
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import pandas as pd
from tqdm import tqdm
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm
import pandas as pd
import random

# Set device for model training and inference
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_sp.csv'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_wp.csv'
# val_data_dir = base_dir + 'val_data.csv'
# target_train_data_dir = base_dir + 'pred_train_data.csv'
# target_val_data_dir = base_dir + 'pred_val_data.csv'
# x_train_tensor_dir = base_dir + 'x_train.pt'
# y_train_tensor_dir = base_dir + 'y_train.pt'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from sklearn.feature_extraction.text import CountVectorizer

# Load Data
csv_data_sp = pd.read_csv(train_data_dir_sp)
csv_data_wp = pd.read_csv(train_data_dir_wp)

# Build Vocabulary
def build_vocabulary(data, max_features=10000):
    vectorizer = CountVectorizer(max_features=max_features, stop_words='english')
    texts = data['question'].tolist() + data['answer'].tolist() + data['distractor1'].tolist() + data['distractor2'].tolist()
    vectorizer.fit(texts)
    return vectorizer

vectorizer = build_vocabulary(pd.concat([csv_data_sp, csv_data_wp]))

# Dataset Preparation
def make_dataset(data, vectorizer):
    x_train = []
    y_train = []
    for idx, row in tqdm(data.iterrows(), total=data.shape[0]):
        q_vec = vectorizer.transform([row['question']]).toarray()
        x_train.append(np.hstack([q_vec, vectorizer.transform([row['answer']]).toarray()]))
        y_train.append(1)
        x_train.append(np.hstack([q_vec, vectorizer.transform([row['distractor1']]).toarray()]))
        y_train.append(0)
        x_train.append(np.hstack([q_vec, vectorizer.transform([row['distractor2']]).toarray()]))
        y_train.append(0)
    return np.array(x_train), np.array(y_train)

x_train_sp, y_train_sp = make_dataset(csv_data_sp, vectorizer)
x_train_wp, y_train_wp = make_dataset(csv_data_wp, vectorizer)

# Convert numpy arrays to tensors
x_train_sp, y_train_sp = torch.FloatTensor(x_train_sp), torch.FloatTensor(y_train_sp).view(-1, 1)
x_train_wp, y_train_wp = torch.FloatTensor(x_train_wp), torch.FloatTensor(y_train_wp).view(-1, 1)

# DataLoader
train_loader_sp = DataLoader(TensorDataset(x_train_sp, y_train_sp), batch_size=32, shuffle=True)
train_loader_wp = DataLoader(TensorDataset(x_train_wp, y_train_wp), batch_size=32, shuffle=True)

# Model Definition
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        input_size = vectorizer.get_feature_names_out().shape[0] * 2
        self.linear1 = nn.Linear(input_size, 768)
        self.dropout1 = nn.Dropout(0.5)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(768, 256)
        self.dropout2 = nn.Dropout(0.5)
        self.linear3 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.linear1(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu(x)
        x = self.linear3(x)
        x = self.sigmoid(x)
        return x.view(-1, 1)

# Initialize and Train Model
model = Model().to(device)
def train(model, loader, epochs):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.BCELoss()
    for epoch in range(epochs):
        total_loss = 0
        for x_batch, y_batch in loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(x_batch)
            loss = loss_fn(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {total_loss / len(loader)}')

train(model, train_loader_sp, 10)
train(model, train_loader_wp, 10)
torch.save(model, '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/Binary-classification-version/bow.pth')


  0%|          | 0/507 [00:00<?, ?it/s]

  0%|          | 0/396 [00:00<?, ?it/s]

Epoch 1, Loss: 0.6637175853053728
Epoch 2, Loss: 0.6197771237542232
Epoch 3, Loss: 0.5613223649561405
Epoch 4, Loss: 0.45661520833770436
Epoch 5, Loss: 0.3444187470401327
Epoch 6, Loss: 0.26402353805800277
Epoch 7, Loss: 0.20032437176754078
Epoch 8, Loss: 0.15611086746988198
Epoch 9, Loss: 0.12726740116098276
Epoch 10, Loss: 0.12097894194691132
Epoch 1, Loss: 0.86847942753842
Epoch 2, Loss: 0.6222567464175978
Epoch 3, Loss: 0.568513307132219
Epoch 4, Loss: 0.5142681543764315
Epoch 5, Loss: 0.4298738680387798
Epoch 6, Loss: 0.33290590110578033
Epoch 7, Loss: 0.28761267387553263
Epoch 8, Loss: 0.23116434169443031
Epoch 9, Loss: 0.2175989027478193
Epoch 10, Loss: 0.18266800055770496


In [2]:
import torch

def inference(model, vectorizer, question, choices):
    model.eval()
    device = next(model.parameters()).device  # Get the device type from the model
    logits = []

    question_vec = vectorizer.transform([question]).toarray()
    for choice in choices:
        # Create a combined vector for question and choice
        combined_vec = np.hstack([question_vec, vectorizer.transform([choice]).toarray()])
        combined_tensor = torch.FloatTensor(combined_vec).to(device)

        # Forward pass to get logits
        logit = model(combined_tensor)
        logits.append(logit.item())

    # Determine which choice had the highest score
    max_index = logits.index(max(logits))
    return choices[max_index]

def predict(model, df, vectorizer):
    predictions = []
    for _, row in df.iterrows():
        question = row['question']
        choices = eval(row['choice_list']) if isinstance(row['choice_list'], str) else row['choice_list']
        predicted_choice = inference(model, vectorizer, question, choices)
        predicted_id = choices.index(predicted_choice) if predicted_choice in choices else -1
        predictions.append(predicted_id)
    df['pred_id'] = predictions
    return df



def write_answer_id(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for i, row in df.iterrows():
            f.write(f"{int(row['pred_id'])}\n")

# Example usage
test_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_sp.csv'
test_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_new_wp.csv'
# Assume model and vectorizer are already initialized
test_df_sp = pd.read_csv(test_dir_sp)
test_df_wp = pd.read_csv(test_dir_wp)

# Predict and handle separately
test_df_sp = predict(model, test_df_sp, vectorizer)
test_df_wp = predict(model, test_df_wp, vectorizer)


test_df_sp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bow_sp.csv")
test_df_wp.to_csv("/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bow_wp.csv")

In [3]:
import pandas as pd

# Load your DataFrame
df_sp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bow_sp.csv')
df_wp = pd.read_csv('/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/predictions_bow_wp.csv')

# Ensure 'id' is treated as string and replace NaN with an empty string if necessary
df_sp['id'] = df_sp['id'].astype(str)
df_wp['id'] = df_wp['id'].astype(str)

# Define function to calculate accuracy
def calculate_accuracy(df, true_label_col, pred_label_col):
    correct_predictions = df[df[true_label_col] == df[pred_label_col]]
    accuracy = len(correct_predictions) / len(df)
    return accuracy

# Calculate accuracies for each subset using the updated string column
S_ori = calculate_accuracy(df_sp[~df_sp['id'].str.contains("_")], 'label', 'pred_id')
S_sem = calculate_accuracy(df_sp[df_sp['id'].str.contains("_SR")], 'label', 'pred_id')
S_con = calculate_accuracy(df_sp[df_sp['id'].str.contains("_CR")], 'label', 'pred_id')
S_ori_sem = calculate_accuracy(df_sp[(df_sp['id'].str.contains("_SR") | ~df_sp['id'].str.contains("_"))], 'label', 'pred_id')
S_ori_sem_con = calculate_accuracy(df_sp, 'label', 'pred_id')  # For all data
S_overall = S_ori_sem_con  # Overall is the same as considering all data

W_ori = calculate_accuracy(df_wp[~df_wp['id'].str.contains("_")], 'label', 'pred_id')
W_sem = calculate_accuracy(df_wp[df_wp['id'].str.contains("_SR")], 'label', 'pred_id')
W_con = calculate_accuracy(df_wp[df_wp['id'].str.contains("_CR")], 'label', 'pred_id')
W_ori_sem = calculate_accuracy(df_wp[(df_wp['id'].str.contains("_SR") | ~df_wp['id'].str.contains("_"))], 'label', 'pred_id')
W_ori_sem_con = calculate_accuracy(df_wp, 'label', 'pred_id')  # For all data
W_overall = S_ori_sem_con  # Overall is the same as considering all data


# Print the calculated scores
print(f'S_ori: {S_ori}')
print(f'S_sem: {S_sem}')
print(f'S_con: {S_con}')
print(f'S_ori_sem: {S_ori_sem}')
print(f'S_ori_sem_con: {S_ori_sem_con}')
print(f'S_overall: {S_overall}')

# Print the calculated scores
print(f'W_ori: {W_ori}')
print(f'W_sem: {W_sem}')
print(f'W_con: {W_con}')
print(f'W_ori_sem: {W_ori_sem}')
print(f'W_ori_sem_con: {W_ori_sem_con}')
print(f'W_overall: {W_overall}')

S_ori: 0.9053254437869822
S_sem: 0.893491124260355
S_con: 0.8816568047337278
S_ori_sem: 0.8994082840236687
S_ori_sem_con: 0.893491124260355
S_overall: 0.893491124260355
W_ori: 0.8181818181818182
W_sem: 0.7727272727272727
W_con: 0.8257575757575758
W_ori_sem: 0.7954545454545454
W_ori_sem_con: 0.8055555555555556
W_overall: 0.893491124260355
